In [16]:
# import library
from sklearn.linear_model import LogisticRegression
import os
import fasttext
import re
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [2]:
# mengambil model word embedding yang digunakan
model = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_10.bin")


In [3]:
# method untuk ubah text menjadi array of words
def change_text_to_array_of_words(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return text

In [4]:
# method untuk ubah dokumen ke array
def change_text_to_array_number(array_of_words):
    # ubah kata menjadi angka
    converted_words = []
    for word in array_of_words:
        wv = model.get_word_vector(word)
        converted_words.append(wv)
    
    # ubah ke numpy array
    converted_words = np.array(converted_words)
    avg_array = np.mean(converted_words, axis=0)
    return avg_array

In [5]:
# method untuk ubah semua file yang ada di directory menjadi data yang siap untuk masuk model LR
def convert_dir_to_input_data(directory):
    files_name = os.listdir(directory)
    list_files = []
    result_array = []

    # buang file DS_Store
    for file in files_name:
        if '.DS_Store' not in file:
            list_files.append(file)

    for file in list_files:
        with open(directory + '/' + file, 'r') as fileNow:
            content = fileNow.read()
            array_words = change_text_to_array_of_words(content)
            avg_array = change_text_to_array_number(array_words)
            if avg_array.shape[0] != 30:
                print(avg_array.shape)
            result_array.append(avg_array)
    result_array = np.array(result_array)
    return result_array

In [10]:
# buat data dan labelnya untuk masuk ke model
dir_asli = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ClassificationModel/Data/Data_asli'
x_asli = convert_dir_to_input_data(dir_asli)

dir_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ClassificationModel/Data/Data_parafrasa'
x_paraphrased = convert_dir_to_input_data(dir_paraphrased)

x = np.concatenate((x_asli, x_paraphrased), axis=0)

zeros_array = np.zeros((1000, 1))  # Create a 3x3 array of zeros
ones_array = np.ones((1000, 1)) 

y = np.concatenate((zeros_array, ones_array), axis=0)


In [13]:
# generate data train dan test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [14]:
# membuat model LR dan melatihnya

model = LogisticRegression(penalty='l2', C=1, max_iter=100, fit_intercept=True, random_state=0, n_jobs=10)
model = model.fit(x_train, y_train)

/Users/williamnehemia/anaconda3/envs/spyder_anaconda_skripsi/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/williamnehemia/anaconda3/envs/spyder_anaconda_skripsi/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
# prediksi model LR dan evaluasi
pred = model.predict(x_test)

score = model.score(x_test, y_test)
print(score)

0.8033333333333333


In [17]:
# save model menggunakan pickle
filename = 'model_lr_testing.sav'
pickle.dump(model, open(filename, 'wb'))

In [18]:
print(pred)

[0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1.
 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1.